In [1]:
import pandas as pd
import numpy as np
import math
import itertools
import plotly.express as px
from tqdm import tqdm
import time
import random
import multiprocessing as mp
import sleepy
from multiprocessing_functions import simulate_schedules, get_ranking

In [2]:
USERNAME = 'TheRealFergus'
YEAR = 2022

In [3]:
# Gets league and owner data
user_id = sleepy.get_user_data(USERNAME)["user_id"]
league_id = sleepy.get_league_ids(user_id, YEAR)[0]
league_raw = sleepy.get_leage(league_id)
owners_raw = sleepy.get_owners(league_id)
owners = owners_raw[["username", "owner_id", "roster_id"]]

In [4]:
# Stores the number of teams in the league
num_teams = owners.shape[0]

# Stores the number of the first week of the league playoffs
playoff_week1 = league_raw["settings"]["playoff_week_start"]

# Maps a unique matchup id corresponding to each combination of two roster ids
matchup_to_roster_id = {matchup[0] + 1: matchup[1] for matchup in 
                        enumerate(itertools.combinations(range(1,13), 2))}

roster_to_matchup_id = {val: key for (key, val) in matchup_to_roster_id.items()}

In [5]:
# Gets regular season matchup data
matchups = (sleepy.get_matchups(league_id, season=True)
            .query(f"starter == True & week < {playoff_week1}")
            .groupby(["week", "roster_id", "matchup_id"])
            [["team_points"]]
            .first()
            .reset_index())

# Merges matchups with owners to include usersernames
matchups = (
    matchups.merge(owners.reset_index(),
                    left_on="roster_id",
                      right_on = "roster_id")
            .assign(matchup_id = matchups["matchup_id"].astype(int))
            .drop(columns="index"))

In [6]:
# Gets regular season matchup data
matchups = (sleepy.get_matchups(league_id, season=True)
            .query(f"starter == True & week < {playoff_week1}")
            .groupby(["week", "roster_id", "matchup_id"])
            [["team_points"]]
            .first()
            .reset_index())

# Merges matchups with owners to include usersernames
matchups = (
    matchups.merge(owners.reset_index(),
                    left_on="roster_id",
                      right_on = "roster_id")
            .assign(matchup_id = matchups["matchup_id"].astype(int))
            .drop(columns="index"))

# Reassigns the matchup id
matchups["matchup_id"] = (
    matchups
    .groupby(["week", "matchup_id"])
    ["roster_id"]
    .transform(lambda x: roster_to_matchup_id[tuple(x.unique())])
    )

# The first week's games
matchups.head(12)

,week,roster_id,matchup_id,team_points,username,owner_id
0,1,1,8,141.13,alecwilson,781258862778015744
1,1,2,13,137.48,namebrant,737201118836346880
2,1,3,29,168.39,therealfergus,871830995287085056
3,1,4,13,88.67,empireyikesback,340376049508429824
4,1,5,40,98.13,pacc,791907251894984704
5,1,6,47,101.86,tonygordzilla22,790423754491678720
6,1,7,40,173.01,mackjyers21,463115290251620352
7,1,8,47,133.25,burgertownthicnred,865421962913157120
8,1,9,8,126.47,thezirconisdragon,865438032692649984
9,1,10,65,113.79,black8yellownation,865844843182694400


In [7]:
# Formats the actual season schedule
season_schedule = tuple(
    tuple(week) for week in matchups.groupby("week")["matchup_id"].agg(set)
)

# Gets the total season points for each owner
owners_points = matchups.groupby("username")[["team_points"]].sum()

In [8]:
# Gets all possible weekly schedules, each tuple contains six matchp id's 
# corresponding to a single game. There are 10395 possible weekly schedules
all_weeks = []
for week in tqdm(itertools.combinations(matchup_to_roster_id.values(), 6), 
                 total = math.comb(66, 6)):
    s = set()
    for match in week:
        s.update(match)
    if len(s) == 12:
        all_weeks.append(tuple(roster_to_matchup_id[match] for match in week))
all_weeks = tuple(all_weeks)

100%|██████████| 90858768/90858768 [00:40<00:00, 2270189.21it/s]


In [9]:
# Maps each week schedule in all possible weeks to a set of weeks. Given that the
# key week appears in a season schedule, none of the weeks in the value set can 
# also appear
similar_weeks = {}
for key_week in tqdm(all_weeks, total = len(all_weeks)):
    similar_weeks[key_week] = (
        set([week for week in all_weeks if len(set(week + key_week)) != 12])
        )

100%|██████████| 10395/10395 [00:23<00:00, 435.33it/s]


In [10]:
# Sets the number of simulations
num_sims_per_process = 10000  # Number of simulations per process
num_processes = 1000  # Number of processes
total_sims = num_sims_per_process * num_processes

In [11]:
# Creates the shared data stuctures to hold results from different processors
manager = mp.Manager()
total_records = manager.dict()
ranking_counts = manager.dict()
for username in owners["username"]:
    total_records[username] = manager.list(np.zeros(12, dtype=int))
# Simulates schedules using multiple processors
if __name__ == "__main__":

    processes = []

    for p in range(num_processes):

        # Continues while at the maximum number of active processes
        while len(processes) >= mp.cpu_count():
            time.sleep(1)
            processes = [p for p in processes if p.is_alive()]

        # When there is less than the max number of processess, start a new one
        process = mp.Process(
            target = simulate_schedules,
            args = (all_weeks, similar_weeks, owners, matchup_to_roster_id, matchups, owners_points, total_records, ranking_counts, num_sims_per_process)
        )
        process.start()
        processes.append(process)

    # Join remaining processes
    for process in processes:
        process.join()

# Formats and stores the results in CSVs
tot_rec = {username: pd.Series(list(lst), index = range(1,13)) 
           for username, lst in dict(total_records).items()}

records_df = (pd.DataFrame(tot_rec)
              .transpose()
              .sort_values(by = list(range(1,13)), ascending = False))

playoffs_df = (pd.DataFrame(dict(ranking_counts), index = ["Count"])
               .transpose()
               .reset_index()
               .query("Count > 0")
               .rename(columns={'level_0': 1, 'level_1': 2, 'level_2': 3,
                                'level_3': 4, 'level_4': 5, 'level_5': 6,
                                'level_6': 7, 'level_7': 8, 'level_8': 9,
                                'level_9': 10, 'level_10': 11, 'level_11': 12})
               .sort_values(by = 'Count', ascending = False))

path_to_data = f'../{YEAR}SimulatedSchedules/data/'

records_df.to_csv(path_to_data + f'{YEAR}_{total_sims}_simulated_records.csv',
                   index_label = 'username')

playoffs_df.to_csv(path_to_data + f'{YEAR}_{total_sims}_simulated_playoffs.csv',
                    index = False)

In [12]:
# Reads previously stored simulated records
records_df = pd.read_csv(
    path_to_data + f'{YEAR}_{total_sims}_simulated_records.csv', 
                         index_col="username")
playoffs_df = pd.read_csv(
    path_to_data + f'{YEAR}_{total_sims}_simulated_playoffs.csv', 
                          index_col = False)

records_df_prop = records_df / (total_sims / 100)
records_df_count = records_df

In [13]:
# Stores the true season ranks
season_ranks = get_ranking(season_schedule, owners, matchup_to_roster_id, matchups, owners_points)
pd.DataFrame(season_ranks)

,rank
burgertownthicnred,1
therealfergus,2
herbietime,3
thezirconisdragon,4
mackjyers21,5
namebrant,6
empireyikesback,7
shakylegs,8
alecwilson,9
black8yellownation,10


In [14]:
records_df_count

,1,2,3,4,5,6,7,8,9,10,11,12
username,,,,,,,,,,,,
therealfergus,7051924,1918803,626791,243497,98729,39456,14863,4442,1246,217,31,1
herbietime,2324007,5870166,1466564,291928,42931,4270,134,0,0,0,0,0
namebrant,182179,549583,1763731,2072461,1714522,1353389,990934,671285,390579,201265,85076,24996
thezirconisdragon,172367,705490,2946337,2284685,1591888,1078057,650100,362141,147320,49836,10448,1331
burgertownthicnred,132582,401293,1322104,1796774,1912706,1705152,1277458,830484,419592,157110,39417,5328
mackjyers21,95071,365016,1130730,1751132,2025363,1891529,1343868,804767,388339,152176,43879,8130
shakylegs,24313,106217,386736,801559,1289289,1799152,2207580,1670266,983890,479109,195368,56521
black8yellownation,14324,64955,250486,481362,755281,1061767,1504236,2038612,2165606,1097009,442847,123515
alecwilson,3060,16875,89890,217731,424854,751722,1320777,2214308,2477538,1489315,729290,264640


In [15]:
# Highlights each user's actual ranking from the season in the simulated dataframe
def highlight_cells(row):
    col_to_highlight = season_ranks[row.name]
    return ['color: red' if int(col) == int(col_to_highlight) 
                            else '' for col in row.index]

highlighted_df = records_df_prop.astype(str).style.apply(highlight_cells,
                                                          axis = 1)
highlighted_df

,1,2,3,4,5,6,7,8,9,10,11,12
username,,,,,,,,,,,,
therealfergus,70.51924,19.18803,6.26791,2.43497,0.98729,0.39456,0.14863,0.04442,0.01246,0.00217,0.00031,1e-05
herbietime,23.24007,58.70166,14.66564,2.91928,0.42931,0.0427,0.00134,0.0,0.0,0.0,0.0,0.0
namebrant,1.82179,5.49583,17.63731,20.72461,17.14522,13.53389,9.90934,6.71285,3.90579,2.01265,0.85076,0.24996
thezirconisdragon,1.72367,7.0549,29.46337,22.84685,15.91888,10.78057,6.501,3.62141,1.4732,0.49836,0.10448,0.01331
burgertownthicnred,1.32582,4.01293,13.22104,17.96774,19.12706,17.05152,12.77458,8.30484,4.19592,1.5711,0.39417,0.05328
mackjyers21,0.95071,3.65016,11.3073,17.51132,20.25363,18.91529,13.43868,8.04767,3.88339,1.52176,0.43879,0.0813
shakylegs,0.24313,1.06217,3.86736,8.01559,12.89289,17.99152,22.0758,16.70266,9.8389,4.79109,1.95368,0.56521
black8yellownation,0.14324,0.64955,2.50486,4.81362,7.55281,10.61767,15.04236,20.38612,21.65606,10.97009,4.42847,1.23515
alecwilson,0.0306,0.16875,0.8989,2.17731,4.24854,7.51722,13.20777,22.14308,24.77538,14.89315,7.2929,2.6464


In [34]:
def get_prob(username, rank, type = "equal"):
    """Gets the probability of a user ranking equal to, worse, or better than 
    they actually did. Probabilites are taken from the simulated rankings.

    Args:
        username (str): the username to get the probability ranking
        rank (int): the rank to determinge
        type (str, optional): Can be "equal", "worse", or "better". For each 
        option. Defaults to "equal".

    Returns:
        str: The function returns the probability of the given user ranking 
        equal/better/or worse than the passed rank. Probabilities are taken from
        the simulated records dataframe.
    """
    user_ix = records_df_prop.index.get_loc(username)
    if type == "equal":
        return records_df_prop.iloc[user_ix, rank-1]
    elif type == "worse":
        return records_df_prop.iloc[user_ix, rank:].sum()
    elif type == "better":
        return records_df_prop.iloc[user_ix, :rank-1].sum()
    elif type == "playoff":
        return records_df_prop.iloc[user_ix, :6].sum()

# Gets the probability dictionaries of worse/equal/better for each user
probs_worse = {username: get_prob(username, rank, type = "worse") for (username, rank) in season_ranks.items()}
probs_better = {username: get_prob(username, rank, type = "better") for (username, rank) in season_ranks.items()}
probs_equal = {username: get_prob(username, rank, type = "equal") for (username, rank) in season_ranks.items()}
probs_playoff = {username: get_prob(username, rank, type = "playoff") for (username, rank) in season_ranks.items()}

# Stores the probabilites of each user's ranking for the season
season_probs = (pd.DataFrame(season_ranks)
                .assign(Better = pd.Series(probs_better))
                .assign(Equal = pd.Series(probs_equal))
                .assign(Worse = pd.Series(probs_worse))
                .assign(Playoff = pd.Series(probs_playoff))
                .reset_index(names="username")
                .set_index("rank")
                .round(3))
season_probs

,username,Better,Equal,Worse,Playoff
rank,,,,,
1,burgertownthicnred,0.000,1.326,98.674,72.706
2,therealfergus,70.519,19.188,10.293,99.792
3,herbietime,81.942,14.666,3.393,99.999
4,thezirconisdragon,38.242,22.847,38.911,87.788
5,mackjyers21,33.419,20.254,46.327,72.588
6,namebrant,62.825,13.534,23.641,76.359
7,empireyikesback,2.594,3.482,93.924,2.594
8,shakylegs,66.148,16.703,17.149,44.073
9,alecwilson,50.392,24.775,24.832,15.041


In [17]:
# Plots the season probabilities as a stacked bar chart
season_probs_melt = pd.melt(season_probs, id_vars=['username', "Playoff"], var_name=' ', value_name='Chance %')
fig = px.bar(
        season_probs_melt, 
        x='username', 
        y='Chance %', 
        color = ' ', 
        title = f'Chance of Ranking for {YEAR} Season',
        color_discrete_sequence = px.colors.qualitative.D3[:3][::-1],
        )

fig.update_layout(width=800,
                  height=400,
                  xaxis_title = None,
                  yaxis_title = "Chance %",
                  title_x = 0.5,
                  font=dict(size=12),
                  margin=dict(t=70, b=75, l=75, r=50),
                  bargap=0.5,
                  legend = dict(title = None,
                                font = dict(size=14))
                 )

In [59]:
# Plots a horizontal bar chart
season_probs_melt = pd.melt(season_probs.reset_index().sort_values(by="rank", ascending=False).set_index("rank"), id_vars=['username', "Playoff"], var_name=' ', value_name='Chance %')

# Create the horizontal bar chart
fig = px.bar(
    season_probs_melt, 
    y='username', 
    x='Chance %', 
    color=' ', 
    title=f'Chance of Ranking for {YEAR} Season',
    orientation='h',  # Horizontal bar chart
    color_discrete_sequence=px.colors.qualitative.D3[:3][::-1],
)

# Update the layout
fig.update_layout(
    width=800,
    height=800,
    xaxis_title="Chance %",
    yaxis_title=None,
    title_x=0.5,
    font=dict(size=12),
    margin=dict(t=70, b=75, l=75, r=50),
    bargap=0.5,
    legend=dict(
        title=None,
        font=dict(size=16)
    )
)

fig.show()

In [60]:
fig.write_html(f'../{YEAR}SimulatedSchedules/output/{YEAR}Probabilities.html', include_plotlyjs='cdn')

In [20]:
playoffs_df

,1,2,3,4,5,6,7,8,9,10,11,12,Count
0,therealfergus,herbietime,thezirconisdragon,namebrant,burgertownthicnred,mackjyers21,shakylegs,alecwilson,black8yellownation,empireyikesback,tonygordzilla22,pacc,16515
1,therealfergus,herbietime,thezirconisdragon,namebrant,burgertownthicnred,mackjyers21,shakylegs,alecwilson,black8yellownation,empireyikesback,pacc,tonygordzilla22,12837
2,therealfergus,herbietime,thezirconisdragon,namebrant,burgertownthicnred,mackjyers21,shakylegs,black8yellownation,alecwilson,empireyikesback,tonygordzilla22,pacc,8987
3,therealfergus,herbietime,namebrant,thezirconisdragon,burgertownthicnred,mackjyers21,shakylegs,alecwilson,black8yellownation,empireyikesback,tonygordzilla22,pacc,7628
4,therealfergus,herbietime,thezirconisdragon,namebrant,burgertownthicnred,mackjyers21,shakylegs,alecwilson,black8yellownation,pacc,empireyikesback,tonygordzilla22,7257
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1013316,therealfergus,burgertownthicnred,herbietime,thezirconisdragon,namebrant,alecwilson,pacc,shakylegs,tonygordzilla22,black8yellownation,mackjyers21,empireyikesback,1
1013317,shakylegs,herbietime,therealfergus,namebrant,burgertownthicnred,thezirconisdragon,mackjyers21,empireyikesback,black8yellownation,pacc,tonygordzilla22,alecwilson,1
1013318,therealfergus,burgertownthicnred,herbietime,alecwilson,shakylegs,mackjyers21,thezirconisdragon,black8yellownation,pacc,namebrant,tonygordzilla22,empireyikesback,1
1013319,herbietime,therealfergus,namebrant,mackjyers21,empireyikesback,pacc,thezirconisdragon,burgertownthicnred,shakylegs,alecwilson,black8yellownation,tonygordzilla22,1


In [21]:
# ge_df_prob contains the chance of each team ranking reater than or equal to 
# the rank achieved
ge_df_prob = pd.DataFrame(playoffs_df)

def better_equal(username, rank):
    return get_prob(username, rank, type = "better") + get_prob(username, rank, type = "equal")

for i in tqdm(range(1,13), total = 12):
    ge_df_prob[str(i)] = playoffs_df[str(i)].apply(lambda x: better_equal(x,i))

100%|██████████| 12/12 [05:46<00:00, 28.88s/it]


In [22]:
# The ranking that occured most often, 16515 times
idx_most_likely_ranking = 1

most_likely_ranking = pd.concat(
    [playoffs_df.iloc[idx_most_likely_ranking,:12].rename("Username"),
     ge_df_prob.iloc[idx_most_likely_ranking,:12].rename("Chance >=")],
     axis=1).reset_index(names="Rank").merge(
         season_probs[["username", "Playoff"]], 
         left_on = "Username",
         right_on = "username"
     ).drop(columns="username")

most_likely_ranking

,Rank,Username,Chance >=,Playoff
0,1,therealfergus,70.51924,99.79
1,2,herbietime,81.94173,100.00
2,3,thezirconisdragon,38.24194,87.79
3,4,namebrant,45.67954,76.36
4,5,burgertownthicnred,55.65459,72.71
5,6,mackjyers21,72.58841,72.59
6,7,shakylegs,66.14846,44.07
7,8,alecwilson,50.39217,15.04
8,9,black8yellownation,83.36629,26.28
9,10,empireyikesback,60.23488,2.59


In [56]:
# The ranking with the smallest combined chance of greater than or equal to in
# the first three rankings
idx_unlikely = ge_df_prob.iloc[:,:3].sum(axis = 1).idxmin()
unlikely_top_3 = (pd.concat(
    [playoffs_df.iloc[idx_unlikely,:12].rename("Username"),
     ge_df_prob.iloc[idx_unlikely,:12].rename("Chance >=")],
     axis = 1)).reset_index(names="Rank").merge(
         season_probs[["username", "Playoff"]], 
         left_on = "Username",
         right_on = "username"
     ).drop(columns="username")
unlikely_top_3

,Rank,Username,Chance >=,Playoff
0,1,burgertownthicnred,1.32582,72.706
1,2,shakylegs,1.30530,44.073
2,3,pacc,0.03931,1.339
3,4,therealfergus,98.41015,99.792
4,5,herbietime,99.95596,99.999
5,6,thezirconisdragon,87.78824,87.788
6,7,mackjyers21,86.02709,72.588
7,8,alecwilson,50.39217,15.041
8,9,black8yellownation,83.36629,26.282
9,10,namebrant,98.89928,76.359


In [24]:
# playoff_df_prob contains the chance of each team making the playoffs
playoff_df_prob = pd.DataFrame(playoffs_df)

def better_equal(username, rank):
    return get_prob(username, rank, type = "playoff")

for i in tqdm(range(1,13), total = 12):
    playoff_df_prob[str(i)] = playoffs_df[str(i)].apply(lambda x: better_equal(x,i))

100%|██████████| 12/12 [04:35<00:00, 22.92s/it]


In [38]:
# The ranking with the lowest combined playoff percentage
idx_unlikely_playoff = playoff_df_prob.iloc[:, :6].product(axis=1).idxmin()
unlikely_playoff_seed2 = (pd.concat(
    [playoffs_df.iloc[idx_unlikely_playoff,:12].rename("Username"),
     ge_df_prob.iloc[idx_unlikely_playoff,:12].rename("Chance >=")],
     axis = 1)).reset_index(names="Rank").merge(
         season_probs[["username", "Playoff"]], 
         left_on = "Username",
         right_on = "username"
     ).drop(columns="username")
unlikely_playoff_seed2

,Rank,Username,Chance >=,Playoff
0,1,herbietime,23.24007,99.999
1,2,therealfergus,89.70727,99.792
2,3,black8yellownation,3.29765,26.282
3,4,empireyikesback,0.33878,2.594
4,5,tonygordzilla22,0.65134,1.440
5,6,pacc,1.33853,1.339
6,7,namebrant,86.26799,76.359
7,8,alecwilson,50.39217,15.041
8,9,thezirconisdragon,99.38385,87.788
9,10,burgertownthicnred,99.55255,72.706


In [26]:
# This cell estimates the total number possible season schedules.

def generate_schedule_nums():
    """This funciton is a modified verstion of the generate schedule function 
    used in the multiprocessing function file. It aims to estimate the total 
    number of possible schedules. 
    
    Generates a 14 week season schedule for an 12 team leage. Each team plays
    all other teams once in the first 11 weeks. The first 3 weeks are repeated
    for the last 3 weeks.

    Returns:
        list: A list of the number of schedules to choose from at each week
    each week.
    """
    weeks = set(all_weeks)
    schedule = []
    num_options = []

    # Recursively fills the weekly schedule at random
    def pick_week(weeks):
        if len(schedule) == 11:
            return
        else:
            tuple_weeks = tuple(weeks)
            num_options.append(len(tuple_weeks))
            choice = random.choice(tuple_weeks)
            schedule.append(choice)
            weeks -= similar_weeks[choice]
            pick_week(weeks)
    
    # The above method has about a 70% success rate to pick a valid yearly 
    # schedule. This while loop will continue until a valid schedule is picked.
    while len(schedule) != 11:
        try:
            pick_week(weeks)
        except IndexError:
            weeks = set(all_weeks)
            schedule = []
            num_options = []

    # Adds the first three weeks of the schedule to the end to finish the 14
    # week season
    schedule += schedule[:3]

    total = 1
    for i in num_options:
        total *= i

    return(total)

np.array([generate_schedule_nums() for i in range(1000)]).mean() * .7

9.978617531734884e+24

In [57]:
# Prints the season outcomes in html
outcome_2022 = owners_raw[["username",
                           "fpts",
                           "fpts_decimal", 
                           "fpts_against", 
                           "fpts_against_decimal", 
                           "wins", 
                           "losses"]]

def format_dec(x):
    out = str((x/100))[1:]
    if len(out) == 2:
        out = out + "0"
    return out

outcome_2022 = (
    outcome_2022
    .assign(points_for = outcome_2022["fpts"].astype(str) 
            + (outcome_2022["fpts_decimal"].apply(format_dec)))
    .assign(points_against = outcome_2022["fpts_against"].astype(str) 
            + (outcome_2022["fpts_against_decimal"].apply(format_dec)))
    .assign(record = outcome_2022["wins"].astype(str) + "-" 
            + outcome_2022["losses"].astype(str))
            )
outcome_2022 = pd.DataFrame(season_ranks).merge(
        outcome_2022,
        left_index=True, 
        right_on='username')


outcome_2022 = (outcome_2022
                .set_index("rank")
                .drop(columns=["fpts", "fpts_decimal", "fpts_against", 
                               "fpts_against_decimal", "wins", "losses"])
                .reset_index()
                .rename(columns={"rank":"Rank",
                                 "username": "Username",
                                 "points_for":"Points For",
                                 "points_against":"Points Against",
                                 "record":"Record"})
)
print(outcome_2022.to_html(index=False))

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Rank</th>
      <th>Username</th>
      <th>Points For</th>
      <th>Points Against</th>
      <th>Record</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>1</td>
      <td>burgertownthicnred</td>
      <td>1856.59</td>
      <td>1599.04</td>
      <td>13-1</td>
    </tr>
    <tr>
      <td>2</td>
      <td>therealfergus</td>
      <td>2176.38</td>
      <td>1684.36</td>
      <td>10-4</td>
    </tr>
    <tr>
      <td>3</td>
      <td>herbietime</td>
      <td>2166.89</td>
      <td>1878.32</td>
      <td>9-5</td>
    </tr>
    <tr>
      <td>4</td>
      <td>thezirconisdragon</td>
      <td>1925.02</td>
      <td>1731.69</td>
      <td>8-6</td>
    </tr>
    <tr>
      <td>5</td>
      <td>mackjyers21</td>
      <td>1838.20</td>
      <td>1870.89</td>
      <td>8-6</td>
    </tr>
    <tr>
      <td>6</td>
      <td>namebrant</td>
      <td>1875.33</td>
      <td>1852.99</td>
      <td>7

In [28]:
# Prints the records count in html
records_df_count.index.name=None
print(records_df_count.astype(str).to_html())

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>1</th>
      <th>2</th>
      <th>3</th>
      <th>4</th>
      <th>5</th>
      <th>6</th>
      <th>7</th>
      <th>8</th>
      <th>9</th>
      <th>10</th>
      <th>11</th>
      <th>12</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>therealfergus</th>
      <td>7051924</td>
      <td>1918803</td>
      <td>626791</td>
      <td>243497</td>
      <td>98729</td>
      <td>39456</td>
      <td>14863</td>
      <td>4442</td>
      <td>1246</td>
      <td>217</td>
      <td>31</td>
      <td>1</td>
    </tr>
    <tr>
      <th>herbietime</th>
      <td>2324007</td>
      <td>5870166</td>
      <td>1466564</td>
      <td>291928</td>
      <td>42931</td>
      <td>4270</td>
      <td>134</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>namebrant</th>
      <td>182179</td>
      <td>549583</td>
      <td

In [29]:
# Prints the highlighted percentage in html
highlighted_df.index.name=None
print(highlighted_df.to_html())

<style type="text/css">
#T_5993b_row0_col1, #T_5993b_row1_col2, #T_5993b_row2_col5, #T_5993b_row3_col3, #T_5993b_row4_col0, #T_5993b_row5_col4, #T_5993b_row6_col7, #T_5993b_row7_col9, #T_5993b_row8_col8, #T_5993b_row9_col10, #T_5993b_row10_col11, #T_5993b_row11_col6 {
  color: red;
}
</style>
<table id="T_5993b">
  <thead>
    <tr>
      <th class="blank level0" >&nbsp;</th>
      <th id="T_5993b_level0_col0" class="col_heading level0 col0" >1</th>
      <th id="T_5993b_level0_col1" class="col_heading level0 col1" >2</th>
      <th id="T_5993b_level0_col2" class="col_heading level0 col2" >3</th>
      <th id="T_5993b_level0_col3" class="col_heading level0 col3" >4</th>
      <th id="T_5993b_level0_col4" class="col_heading level0 col4" >5</th>
      <th id="T_5993b_level0_col5" class="col_heading level0 col5" >6</th>
      <th id="T_5993b_level0_col6" class="col_heading level0 col6" >7</th>
      <th id="T_5993b_level0_col7" class="col_heading level0 col7" >8</th>
      <th id="T_5993b_

In [30]:
# Prints season_probs in html
print(season_probs.reset_index().rename(columns={"rank":"Rank",
                                                 "username":"Username"})
                                                 .to_html(index=False))

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Rank</th>
      <th>Username</th>
      <th>Better</th>
      <th>Equal</th>
      <th>Worse</th>
      <th>Playoff</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>1</td>
      <td>burgertownthicnred</td>
      <td>0.00</td>
      <td>1.33</td>
      <td>98.67</td>
      <td>72.71</td>
    </tr>
    <tr>
      <td>2</td>
      <td>therealfergus</td>
      <td>70.52</td>
      <td>19.19</td>
      <td>10.29</td>
      <td>99.79</td>
    </tr>
    <tr>
      <td>3</td>
      <td>herbietime</td>
      <td>81.94</td>
      <td>14.67</td>
      <td>3.39</td>
      <td>100.00</td>
    </tr>
    <tr>
      <td>4</td>
      <td>thezirconisdragon</td>
      <td>38.24</td>
      <td>22.85</td>
      <td>38.91</td>
      <td>87.79</td>
    </tr>
    <tr>
      <td>5</td>
      <td>mackjyers21</td>
      <td>33.42</td>
      <td>20.25</td>
      <td>46.33</td>
      <td>72.59</td>
    </tr>
    <tr>

In [31]:
# Prints most_likely_ranking in html
print(most_likely_ranking.to_html(index=False))

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Rank</th>
      <th>Username</th>
      <th>Chance &gt;=</th>
      <th>Playoff</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>1</td>
      <td>therealfergus</td>
      <td>70.51924</td>
      <td>99.79</td>
    </tr>
    <tr>
      <td>2</td>
      <td>herbietime</td>
      <td>81.94173</td>
      <td>100.00</td>
    </tr>
    <tr>
      <td>3</td>
      <td>thezirconisdragon</td>
      <td>38.24194</td>
      <td>87.79</td>
    </tr>
    <tr>
      <td>4</td>
      <td>namebrant</td>
      <td>45.67954</td>
      <td>76.36</td>
    </tr>
    <tr>
      <td>5</td>
      <td>burgertownthicnred</td>
      <td>55.65459</td>
      <td>72.71</td>
    </tr>
    <tr>
      <td>6</td>
      <td>mackjyers21</td>
      <td>72.58841</td>
      <td>72.59</td>
    </tr>
    <tr>
      <td>7</td>
      <td>shakylegs</td>
      <td>66.14846</td>
      <td>44.07</td>
    </tr>
    <tr>
      <td>8</td

In [32]:
# Prints unlikely_top_3 in html
print(unlikely_top_3.to_html(index=False))

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Rank</th>
      <th>Username</th>
      <th>Chance &gt;=</th>
      <th>Playoff</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>1</td>
      <td>burgertownthicnred</td>
      <td>1.32582</td>
      <td>72.71</td>
    </tr>
    <tr>
      <td>2</td>
      <td>shakylegs</td>
      <td>1.30530</td>
      <td>44.07</td>
    </tr>
    <tr>
      <td>3</td>
      <td>pacc</td>
      <td>0.03931</td>
      <td>1.34</td>
    </tr>
    <tr>
      <td>4</td>
      <td>therealfergus</td>
      <td>98.41015</td>
      <td>99.79</td>
    </tr>
    <tr>
      <td>5</td>
      <td>herbietime</td>
      <td>99.95596</td>
      <td>100.00</td>
    </tr>
    <tr>
      <td>6</td>
      <td>thezirconisdragon</td>
      <td>87.78824</td>
      <td>87.79</td>
    </tr>
    <tr>
      <td>7</td>
      <td>mackjyers21</td>
      <td>86.02709</td>
      <td>72.59</td>
    </tr>
    <tr>
      <td>8</td>
      <

In [33]:
# Prints unlikely_playoff_seed2 in html
print(unlikely_playoff_seed2.to_html(index=False))

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Rank</th>
      <th>Username</th>
      <th>Chance &gt;=</th>
      <th>Playoff</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>1</td>
      <td>herbietime</td>
      <td>23.24007</td>
      <td>100.00</td>
    </tr>
    <tr>
      <td>2</td>
      <td>therealfergus</td>
      <td>89.70727</td>
      <td>99.79</td>
    </tr>
    <tr>
      <td>3</td>
      <td>black8yellownation</td>
      <td>3.29765</td>
      <td>26.28</td>
    </tr>
    <tr>
      <td>4</td>
      <td>empireyikesback</td>
      <td>0.33878</td>
      <td>2.59</td>
    </tr>
    <tr>
      <td>5</td>
      <td>tonygordzilla22</td>
      <td>0.65134</td>
      <td>1.44</td>
    </tr>
    <tr>
      <td>6</td>
      <td>pacc</td>
      <td>1.33853</td>
      <td>1.34</td>
    </tr>
    <tr>
      <td>7</td>
      <td>namebrant</td>
      <td>86.26799</td>
      <td>76.36</td>
    </tr>
    <tr>
      <td>8</td>
      <t